In [1]:
import json
from typing import Dict, Any

from anthropic import Anthropic
from anthropic.types import ToolParam

In [7]:
# Load environment variables
from helper import load_env
load_env()

MODEL_NAME="claude-3-5-sonnet-20241022"
client = Anthropic()

In [3]:
# Tool 1: Calculator
def calculator(operation: str, a: float, b: float) -> Dict[str, Any]:
    """Simple calculator function"""
    operations = {
        "add": a + b,
        "subtract": a - b,
        "multiply": a * b,
        "divide": a / b if b != 0 else "Error: Division by zero"
    }
    return {
        "operation": operation,
        "numbers": [a, b],
        "result": operations.get(operation, "Unknown operation")
    }

# Tool 2: Weather
def get_weather(city: str) -> Dict[str, Any]:
    """Get weather for a city (mock data)"""
    weather_data = {
        "paris": {"temp": "17°C", "condition": "Cloudy"},
        "london": {"temp": "15°C", "condition": "Rainy"},
        "tokyo": {"temp": "25°C", "condition": "Sunny"},
        "new york": {"temp": "20°C", "condition": "Partly cloudy"}
    }
    
    city_key = city.lower()
    if city_key in weather_data:
        return {"city": city, **weather_data[city_key]}
    return {"city": city, "error": "Weather data not available"}

# Tool 3: User lookup
def get_user_info(user_id: int) -> Dict[str, Any]:
    """Get user information by ID"""
    users = {
        1: {"name": "Alice", "email": "alice@email.com", "city": "Paris"},
        2: {"name": "Bob", "email": "bob@email.com", "city": "London"},
        3: {"name": "Charlie", "email": "charlie@email.com", "city": "Tokyo"}
    }
    
    if user_id in users:
        return {"user_id": user_id, **users[user_id]}
    return {"user_id": user_id, "error": "User not found"}

In [4]:
# Define tool schemas
calculator_tool: ToolParam  = {
    "name": "calculator",
    "description": "Perform basic math calculations",
    "input_schema": {
        "type": "object",
        "properties": {
            "operation": {"type": "string", "enum": ["add", "subtract", "multiply", "divide"]},
            "a": {"type": "number"},
            "b": {"type": "number"}
        },
        "required": ["operation", "a", "b"]
    }
}

weather_tool: ToolParam = {
    "name": "get_weather",
    "description": "Get weather information for a city",
    "input_schema": {
        "type": "object",
        "properties": {
            "city": {"type": "string"}
        },
        "required": ["city"]
    }
}

user_tool: ToolParam = {
    "name": "get_user_info",
    "description": "Get user information by ID",
    "input_schema": {
        "type": "object",
        "properties": {
            "user_id": {"type": "integer"}
        },
        "required": ["user_id"]
    }
}

In [5]:
# Tool function registry
CUSTOM_TOOLS = {
    "calculator": calculator,
    "get_weather": get_weather,
    "get_user_info": get_user_info
}


In [8]:
def simple_conversation_with_tools(user_message: str):
    """Simple conversation loop with multiple tools"""
    
    print(f"User: {user_message}")
    
    # Available tools with proper typing
    all_tools: list[ToolParam] = [calculator_tool, weather_tool, user_tool]
    
    # Start conversation
    messages = [{"role": "user", "content": user_message}]
    
    
    while True:       
        # Get Claude's response
        response = client.messages.create(
            model=MODEL_NAME,
            max_tokens=1500,
            messages=messages,
            tools=all_tools
        )
        
        # Add Claude's response to messages
        messages.append({"role": "assistant", "content": response.content})
        
        # Check what Claude wants to do
        text_parts = []
        tool_calls = []
        
        for content in response.content:
            if content.type == "text":
                text_parts.append(content.text)
            elif content.type == "tool_use":
                tool_calls.append(content)
        
        # Show Claude's text response
        if text_parts:
            claude_text = " ".join(text_parts)
            print(f"Claude: {claude_text}")
        
        # If no tools needed, we're done
        if not tool_calls:
            print("\n✅ Conversation complete!")
            return " ".join(text_parts)
        
        # Execute tools
        print(f"\n🔧 Claude wants to use {len(tool_calls)} tool(s):")
        
        tool_results = []
        for tool_call in tool_calls:
            tool_name = tool_call.name
            tool_input = tool_call.input
            tool_id = tool_call.id
            
            print(f"  - {tool_name}: {tool_input}")
            
            try:
                # Call the tool function
                result = CUSTOM_TOOLS[tool_name](**tool_input)
                
                tool_results.append({
                    "type": "tool_result",
                    "tool_use_id": tool_id,
                    "content": json.dumps(result)
                })
                
                print(f"    ✅ Result: {result}")
                
            except Exception as e:
                print(f"    ❌ Error: {e}")
                tool_results.append({
                    "type": "tool_result",
                    "tool_use_id": tool_id,
                    "content": json.dumps({"error": str(e)})
                })
        
        # Add tool results to conversation
        messages.append({"role": "user", "content": tool_results})


In [9]:
# Example 1: Simple calculation
def example_1():
    print("="*50)
    print("EXAMPLE 1: Simple Calculation")
    print("="*50)
    
    result = simple_conversation_with_tools("What's 25 + 17?")
    print(f"\nFinal answer: {result}")

In [10]:
example_1()

EXAMPLE 1: Simple Calculation
User: What's 25 + 17?
Claude: I'll help you calculate 25 + 17 using the calculator function.

🔧 Claude wants to use 1 tool(s):
  - calculator: {'operation': 'add', 'a': 25, 'b': 17}
    ✅ Result: {'operation': 'add', 'numbers': [25, 17], 'result': 42}
Claude: 25 + 17 = 42

✅ Conversation complete!

Final answer: 25 + 17 = 42


In [11]:
# Example 2: Weather lookup
def example_2():
    print("\n" + "="*50)
    print("EXAMPLE 2: Weather Lookup")
    print("="*50)
    
    result = simple_conversation_with_tools("What's the weather like in Paris?")
    print(f"\nFinal answer: {result}")

In [12]:
example_2()


EXAMPLE 2: Weather Lookup
User: What's the weather like in Paris?
Claude: I'll help you check the weather in Paris using the get_weather function.

🔧 Claude wants to use 1 tool(s):
  - get_weather: {'city': 'Paris'}
    ✅ Result: {'city': 'Paris', 'temp': '17°C', 'condition': 'Cloudy'}
Claude: According to the weather information, it is currently 17°C (Celsius) and cloudy in Paris.

✅ Conversation complete!

Final answer: According to the weather information, it is currently 17°C (Celsius) and cloudy in Paris.


In [13]:
# Example 3: Multiple tools in sequence
def example_3():
    print("\n" + "="*50)
    print("EXAMPLE 3: Multiple Tools")
    print("="*50)
    
    result = simple_conversation_with_tools(
        "Get user 1's information, then tell me the weather in their city, then calculate their user_id * 10"
    )
    print(f"\nFinal answer: {result}")

In [14]:
example_3()


EXAMPLE 3: Multiple Tools
User: Get user 1's information, then tell me the weather in their city, then calculate their user_id * 10
Claude: I'll help you with that sequence of operations. Let's break it down into steps:

1. First, let's get user 1's information:

🔧 Claude wants to use 1 tool(s):
  - get_user_info: {'user_id': 1}
    ✅ Result: {'user_id': 1, 'name': 'Alice', 'email': 'alice@email.com', 'city': 'Paris'}
Claude: 2. Now that we know Alice lives in Paris, let's get the weather there:

🔧 Claude wants to use 1 tool(s):
  - get_weather: {'city': 'Paris'}
    ✅ Result: {'city': 'Paris', 'temp': '17°C', 'condition': 'Cloudy'}
Claude: 3. Finally, let's multiply their user_id (1) by 10:

🔧 Claude wants to use 1 tool(s):
  - calculator: {'operation': 'multiply', 'a': 1, 'b': 10}
    ✅ Result: {'operation': 'multiply', 'numbers': [1, 10], 'result': 10}
Claude: Here's a summary of what we found:
- User 1 is Alice, who lives in Paris
- The weather in Paris is currently 17°C and Cloud

In [15]:
def example_4():
    print("\n" + "="*50)
    print("EXAMPLE 4: Complex Request")
    print("="*50)
    
    result = simple_conversation_with_tools(
        "I need: weather for Tokyo and London, calculate 100 * 1.5, and get info for user 2"
    )
    print(f"\nFinal answer: {result}")

In [16]:
example_4()


EXAMPLE 4: Complex Request
User: I need: weather for Tokyo and London, calculate 100 * 1.5, and get info for user 2
Claude: I'll help you with those requests using the available tools.

1. Weather for Tokyo:

🔧 Claude wants to use 1 tool(s):
  - get_weather: {'city': 'Tokyo'}
    ✅ Result: {'city': 'Tokyo', 'temp': '25°C', 'condition': 'Sunny'}
Claude: 2. Weather for London:

🔧 Claude wants to use 1 tool(s):
  - get_weather: {'city': 'London'}
    ✅ Result: {'city': 'London', 'temp': '15°C', 'condition': 'Rainy'}
Claude: 3. Calculate 100 * 1.5:

🔧 Claude wants to use 1 tool(s):
  - calculator: {'operation': 'multiply', 'a': 100, 'b': 1.5}
    ✅ Result: {'operation': 'multiply', 'numbers': [100, 1.5], 'result': 150.0}
Claude: 4. Get info for user 2:

🔧 Claude wants to use 1 tool(s):
  - get_user_info: {'user_id': 2}
    ✅ Result: {'user_id': 2, 'name': 'Bob', 'email': 'bob@email.com', 'city': 'London'}
Claude: Here's a summary of all the results:
- Tokyo: 25°C, Sunny
- London: 15°C, Ra